In [42]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
from src.data_processing.constants import (
    KAGGLE_DATA_PATH,
    KAGGLE_OUTPUT_DATA_PATH,
    LOCAL_DATA_PATH,
    LOCAL_OUTPUT_DATA_PATH,
)

IS_KAGGLE = False
DATA_DIR = KAGGLE_DATA_PATH if IS_KAGGLE else LOCAL_DATA_PATH
OUTPUT_DIR = KAGGLE_OUTPUT_DATA_PATH if IS_KAGGLE else LOCAL_OUTPUT_DATA_PATH

In [ ]:
from src.data_processing.data_utils import prepare_df

new_df = prepare_df(data_directory=DATA_DIR)
new_df.head()

In [ ]:
from src.data_processing.data_utils import load_img, id2mask, show_img
from matplotlib import pyplot as plt

row = 1
col = 4
plt.figure(figsize=(5 * col, 5 * row))
random_slices_to_plot = (
    new_df[~new_df.segmentation.isna()].sample(frac=1.0)["id"].unique()[: row * col]
)
for i, id_ in enumerate(random_slices_to_plot):
    img = load_img(new_df[new_df["id"] == id_].image_path.iloc[0])
    mask = id2mask(id_, df=new_df) * 255
    plt.subplot(row, col, i + 1)
    show_img(img, mask=mask)
    plt.tight_layout()

In [ ]:
from src.data_processing.data_utils import save_mask

save_mask(id_="case123_day22_slice_0117", df=new_df, save_dir=OUTPUT_DIR)

In [ ]:
import numpy as np

img = load_img(new_df[new_df["id"] == "case123_day22_slice_0117"].iloc[0].image_path)
mask_path = OUTPUT_DIR / "np" / "case123_day22_slice_0117.npy"
print(mask_path)
mask = np.load(str(mask_path))
plt.figure(figsize=(5, 5))
show_img(img, mask)

In [ ]:
from tqdm import tqdm
from joblib import delayed, Parallel

ids = new_df["id"].unique()
_ = Parallel(n_jobs=-1, backend="threading")(
    delayed(save_mask)(id_) for id_ in tqdm(ids, total=len(ids))
)

In [ ]:
new_df["mask_path"] = str(OUTPUT_DIR) + "/png/" + new_df.id + ".png"
new_df.to_csv(str(OUTPUT_DIR / "train.csv"), index=False)

In [ ]:
import shutil

if IS_KAGGLE:
    shutil.make_archive(
        "/kaggle/working/png",
        "zip",
        str(OUTPUT_DIR / "png"),
        "uw-madison-gi-tract-image-segmentation",
    )
    shutil.make_archive(
        "/kaggle/working/np",
        "zip",
        str(OUTPUT_DIR / "np"),
        "uw-madison-gi-tract-image-segmentation",
    )